In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

spectrum_train = pd.read_excel("spectrum_train.xlsx")
spectrum_valid = pd.read_excel("spectrum_valid.xlsx")
temp_train = pd.read_excel("temp_train.xlsx")
temp_valid = pd.read_excel("temp_valid.xlsx")

In [3]:
X_train = spectrum_train
y_train = temp_train
X_test = spectrum_valid
y_test = temp_valid

In [6]:
import torch
import torch.nn as nn

class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.fc1 = nn.Linear(66, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 11)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the network
net = MyNetwork()

# Generate a random input tensor with size (batch_size, input_size)
input_tensor = torch.tensor(X_train.values)

# Pass the input tensor through the network
output_tensor = net(input_tensor)

print(output_tensor.shape)  # (32, 66)


RuntimeError: mat1 and mat2 must have the same dtype